# Day 1

## Part 1

In [6]:
with open('inputs/day1.txt') as f:
    s = f.read()[:-1].split('\n')
    
sum([int(n) for n in s])

525

## Part 2

In [10]:
seen = set()

frequencies = [int(n) for n in s]

i = 0
f = 0
while f not in seen:
    seen.add(f)
    f += frequencies[i%len(frequencies)]
    i += 1
    
f

75749

# Day 2

## Part 1

In [21]:
with open('inputs/day2.txt') as f:
    s = f.read()[:-1].split('\n')

twos = 0
threes = 0
for box in s:
    two = False
    three = False
    for c in range(ord('a'), ord('z')+1):
        count = box.count(chr(c))
        if count == 2:
            two = True
        if count == 3:
            three = True
    if two:
        twos += 1
    if three:
        threes += 1
            
twos*threes

7350

## Part 2

In [25]:
def hamming(s1,s2):
    d = 0
    for i in range(len(s1)):
        if s1[i] != s2[i]:
            d += 1
    return d

for i in range(len(s)):
    for j in range(i+1,len(s)):
        if hamming(s[i],s[j]) == 1:
            s1 = s[i]
            s2 = s[j]
            
''.join([s1[i] for i in range(len(s1)) if s1[i] == s2[i]])

'wmlnjevbfodamyiqpucrhsukg'

# Day 3

## Part 1

In [45]:
with open('inputs/day3.txt') as f:
    s = f.read()[:-1].split('\n')
    
def parse(claim):
    pos = claim.split('@ ')[1].split(':')[0]
    pos = (int(pos.split(',')[0]), int(pos.split(',')[1]))
    size = claim.split(': ')[-1]
    size = (int(size.split('x')[0]), int(size.split('x')[1]))
    return (pos, (pos[0]+size[0], pos[1]+size[1]))
    
claims = {parse(c): c.split(' ')[0][1:] for c in s}
claimed = set()
double_claimed = set()

for c in claims:
    for x in range(c[0][0], c[1][0]):
        for y in range(c[0][1], c[1][1]):
            if (x,y) in claimed:
                double_claimed.add((x,y))
            else:
                claimed.add((x,y))
            
len(double_claimed)

111935

## Part 2

In [46]:
for c in claims:
    match = True
    for x in range(c[0][0], c[1][0]):
        for y in range(c[0][1], c[1][1]):
            if (x,y) in double_claimed:
                match = False
    if match:
        break
        
claims[c]

'650'

# Day 4

## Part 1

In [70]:
with open('inputs/day4.txt') as f:
    s = f.read()[:-1].split('\n')
    
s.sort()
current_guard = None
last_sleep = None
last_wake = None
sleeping = False
shifts = {}

def parse(entry):
    global current_guard
    global last_sleep
    global last_wake
    global sleeping
    
    minute = int(entry.split(':')[1].split(']')[0])
    hour = int(entry.split(' ')[1].split(':')[0])
    if hour != 0:
        minute = 0
    if 'Guard' in entry:
        if sleeping:
            for t in range(last_sleep, 60):
                shifts[current_guard][len(shifts[current_guard])-1].append(t)
        sleeping = False
        
        current_guard = int(entry.split('#')[1].split(' ')[0])
        if current_guard not in shifts:
            shifts[current_guard] = {}
        shifts[current_guard][len(shifts[current_guard])] = []
    else:
        if 'asleep' in entry:
            last_sleep = minute
            sleeping = True
        else:
            sleeping = False
            last_wake = minute
            for t in range(last_sleep, last_wake):
                shifts[current_guard][len(shifts[current_guard])-1].append(t)
        
for entry in s:
    parse(entry)
    
sleeps = {guard: sum([len(shifts[guard][s]) for s in shifts[guard]]) for guard in shifts}
most_sleeping = max(sleeps, key=sleeps.get)

sleep_minutes = []
for shift in shifts[most_sleeping]:
    sleep_minutes += shifts[most_sleeping][shift]

minute_sleep_count = [sleep_minutes.count(t) for t in range(60)]
max_slept_minute = max(minute_sleep_count)
most_slept_minute = minute_sleep_count.index(max_slept_minute)

most_sleeping*most_slept_minute

8421

## Part 2

In [75]:
frequencies = {}

for guard in shifts:
    sleep_minutes = []
    for shift in shifts[guard]:
        sleep_minutes += shifts[guard][shift]
    frequencies[guard] = [sleep_minutes.count(t) for t in range(60)]
    
max_guard_minute = 0
guard = 0
minute = 0
for g in frequencies:
    for m in range(60):
        if frequencies[g][m] > max_guard_minute:
            max_guard_minute = frequencies[g][m]
            guard = g
            minute = m
            
guard*minute

83359

# Day 5

## Part 1

In [108]:
with open('inputs/day5.txt') as f:
    s = f.read()[:-1]

units = [chr(c) for c in range(ord('a'), ord('z')+1)]

def react(s):
    start = ''
    result = s
    while len(start) != len(result):
        start = result
        for u in units:
            while u + u.upper() in result:
                result = result.replace(u + u.upper(), '')
            while u.upper() + u in result:
                result = result.replace(u.upper() + u, '')
    return result
    
len(react(s))

11298

## Part 2

In [109]:
min_length = len(s)
for u in units:
    poly = ''.join([c for c in s if c != u and c != u.upper()])
    result = react(poly)
    if len(result) < min_length:
        min_length = len(result)
        
min_length

5148

# Day 6

## Part 1

In [24]:
import queue

with open('inputs/day6.txt') as f:
    s = f.read()[:-1].split('\n')
    
coords = {i: (int(s[i].split(', ')[0]), int(s[i].split(', ')[1])) for i in range(len(s))}
closest = {}

limit = 10000

def distances(coord):    
    return {c: abs(coords[c][0]-coord[0]) + abs(coords[c][1]-coord[1]) for c in coords}

def closest(coord):    
    d = distances(coord)
    min_d = min(d.values())
    d = {c: d[c] for c in d if d[c] == min_d}
    return next(iter(d)) if len(d) == 1 else -1
    
def bfs(source, coord, visited, matching, q):
    if coord in visited:
        return
    
    visited.add(coord)
    
    if closest(coord) == source:
        matching.add(coord)

        (x,y) = coord
        if (x,y-1) not in visited:
            q.put((x,y-1))
        if (x,y+1) not in visited:
            q.put((x,y+1))
        if (x-1,y) not in visited:
            q.put((x-1,y))
        if (x+1,y) not in visited:
            q.put((x+1,y))
            
def discover(source):
    q = queue.Queue()
    matching = set()
    visited = set()
    q.put(coords[source])
    while not q.empty() and len(matching) < limit:
        bfs(source, q.get(), visited, matching, q)
    if not q.empty():
        return -1
    else:
        return len(matching)
    
areas = {c: discover(c) for c in coords}

max(areas.values())

4060

## Part 2

In [32]:
matches = 0
for x in range(500):
    for y in range(500):
        if sum(distances((x,y)).values()) < 10000:
            matches += 1
            
matches

36136

# Day 7

## Part 1

In [48]:
with open('inputs/day7.txt') as f:
    s = f.read()[:-1].split('\n')

deps = [(d.split(' ')[1], d.split(' ')[-3]) for d in s]
    
dependencies = {}
for dep in deps:
    if dep[0] not in dependencies:
        dependencies[dep[0]] = []
    if dep[1] not in dependencies:
        dependencies[dep[1]] = []
    dependencies[dep[1]].append(dep[0])
    
order = []

while len(dependencies) != len(order):
    executable = [d for d in dependencies if len(dependencies[d]) == 0 and d not in order]
    executable.sort()
    e = executable[0]
    order.append(e)
    dependencies = {d: [d1 for d1 in dependencies[d] if d1!= e] for d in dependencies}
    
''.join(order)

'FMOXCDGJRAUIHKNYZTESWLPBQV'

## Part 2

In [60]:
finish_times = {}
startable_time = {}
free_workers = 5

dependencies = {}
for dep in deps:
    if dep[0] not in dependencies:
        dependencies[dep[0]] = []
    if dep[1] not in dependencies:
        dependencies[dep[1]] = []
    dependencies[dep[1]].append(dep[0])

def task_time(task_id):
    return ord(task_id) - 4

def free_worker():
    global free_workers
    free_workers = min(free_workers+1, 5)
    
def tick(t):
    global dependencies
    finished = [f for f in finish_times if finish_times[f] == t]
    for f in finished:
        free_worker()
        dependencies = {d: [d1 for d1 in dependencies[d] if d1!= f] for d in dependencies}
    executable = [d for d in dependencies if len(dependencies[d]) == 0 and d not in finish_times]
    executable.sort()
    for i in range(min(free_workers, len(executable))):
        e = executable[0]
        executable = executable[1:]
        finish_times[e] = t + task_time(e)
        
        
t = 0
while len(dependencies) != len(finish_times):
    tick(t)
    t += 1
    
max(finish_times.values())

1053

# Day 8

## Part 1

In [60]:
with open('inputs/day8.txt') as f:
    s = f.read()[:-1]
    
tree = [int(n) for n in s.split(' ')]

class Node:
    def __init__(self, parent=None):
        self.parent = parent
        self.children = []
        self.metadata = []
        
    def meta_sum(self):
        a = sum(self.metadata)
        for child in self.children:
            a += child.meta_sum()
        return a
    
    def value(self):
        if len(self.children) == 0:
            return sum(self.metadata)
        else:
            a = 0
            for m in self.metadata:
                if m <= len(self.children):
                    a += self.children[m-1].value()
            return a
                
        
root = Node()
current = root
offset = 0
while len(tree) > 0:
    while tree[offset] != 0:
        offset += 2
        current.children.append(Node(current))
        current = current.children[-1]
        
    for i in range(offset+2, offset+2+tree[offset+1]):
        current.metadata.append(tree[i])
        
    tree = tree[:offset] + tree[offset+2+tree[offset+1]:]
    
    if offset != 0:
        offset -= 2
        tree[offset] -= 1
        current = current.parent
        
root.meta_sum()

46096

## Part 2

In [61]:
root.value()

24820

# Day 9

## Part 1

In [69]:
elves = 416
max_marble = 71617

class Node:
    def __init__(self, value, prev_node = None, next_node = None):
        self.value = value
        if prev_node is not None:
            self.prev = prev_node
            prev_node.next = self
        else:
            self.prev = self
        if next_node is not None:
            self.next = next_node
            next_node.prev = self
        else:
            self.next = self
        
    def pick(self):
        self.prev.next = self.next
        self.next.prev = self.prev
        return self.value

players = {i: 0 for i in range(elves)}
current = Node(0)

for i in range(0, max_marble):
    if (i+1) % 23 != 0:
        current = current.next
        current = Node(i+1, current, current.next)
    else:
        players[i%elves] += i+1
        for j in range(7):
            current = current.prev
        players[i%elves] += current.pick()
        current = current.next
        
max(players.values())

436720

## Part 2

In [72]:
max_marble = 7161700

players = {i: 0 for i in range(elves)}
current = Node(0)

for i in range(0, max_marble):
    if (i+1) % 23 != 0:
        current = current.next
        current = Node(i+1, current, current.next)
    else:
        players[i%elves] += i+1
        for j in range(7):
            current = current.prev
        players[i%elves] += current.pick()
        current = current.next
        
max(players.values())

3527845091

# Day 10

## Part 1

In [121]:
with open('inputs/day10.txt') as f:
    s = f.read()[:-1].split('\n')
    
def parse(string):
    numbers = string[:-1].replace('position=<', '').replace('> velocity=<', ', ').split(', ')
    return ((int(numbers[0]), int(numbers[1])),(int(numbers[2]), int(numbers[3])))

stars = [(parse(star)[0], parse(star)[1])  for star in s]

def move(stars):
    return [((star[0][0] + star[1][0], star[0][1] + star[1][1]), star[1]) for star in stars]

def draw(stars):
    stars = {s[0] for s in stars}
    min_x = min_y = max_x = max_y = None
    for star in stars:
        if min_x is None or min_x > star[0]:
            min_x = star[0]
        if max_x is None or max_x < star[0]:
            max_x = star[0]
        if min_y is None or min_y > star[1]:
            min_y = star[1]
        if max_y is None or max_y < star[1]:
            max_y = star[1]
    
    return [''.join(['#' if (x,y) in stars else '.' for x in range(min_x, max_x+1)]) for y in range(min_y, max_y+1)]

pattern_size = 6
def check_for_message(stars):
    stars = {s[0] for s in stars}
    for star in stars:
        (x,y) = star
        
        match_x = True
        match_y = True
        
        for i in range(1,pattern_size+1):
            if match_x:
                match_x = (x+i, y) in stars
            if match_y:
                match_y = (x, y+i) in stars
            if not (match_x or match_y):
                break
        
        if match_x or match_y:
            return True
    return False
    
while not check_for_message(stars):
    stars = move(stars)

draw(stars)

['#####...#####...#....#..#....#..#....#..######..######..#####.',
 '#....#..#....#..##...#..##...#..#....#..#............#..#....#',
 '#....#..#....#..##...#..##...#...#..#...#............#..#....#',
 '#....#..#....#..#.#..#..#.#..#...#..#...#...........#...#....#',
 '#####...#####...#.#..#..#.#..#....##....#####......#....#####.',
 '#..#....#.......#..#.#..#..#.#....##....#.........#.....#..#..',
 '#...#...#.......#..#.#..#..#.#...#..#...#........#......#...#.',
 '#...#...#.......#...##..#...##...#..#...#.......#.......#...#.',
 '#....#..#.......#...##..#...##..#....#..#.......#.......#....#',
 '#....#..#.......#....#..#....#..#....#..#.......######..#....#']

## Part 2

In [122]:
stars = [(parse(star)[0], parse(star)[1])  for star in s]

moves = 0
while not check_for_message(stars):
    moves += 1
    stars = move(stars)
    
moves

10946

# Day 11

## Part 1

In [145]:
sn = 3031

def power(x,y):
    rack_id = x+10
    power_level = rack_id * y
    power_level += sn
    power_level *= rack_id
    power_level //= 100
    power_level %= 10
    power_level -= 5
    return power_level
    
cells = {(x,y): power(x,y) for y in range(1,301) for x in range(1,301)}
cache = {}

def area_power(x,y,size):
    if (x,y,size) in cache:
        return cache[(x,y,size)]
    if size == 1:
        return cells[(x,y)]
    power = area_power(x,y,size-1)
    for i in range(size):
        power += cells[(x+size-1,y+i)]
        power += cells[(x+i,y+size-1)]
    power -= cells[(x+size-1,y+size-1)]
            
    cache[(x,y,size)] = power
    return power

def max_coords(size):
    max_power = -1000000
    max_x = max_y = 0
    for x in range(1,301-(size-1)):
        for y in range(1, 301-(size-1)):
            power = area_power(x,y,size)
            if power > max_power:
                max_power = power
                max_x = x
                max_y = y
    return max_power, max_x, max_y


max_coords(3)[1:]

(21, 76)

## Part 2

In [159]:
max_power = -1000000
max_x = max_y = max_size = 0
for size in range(1, 301):
    power,x,y = max_coords(size)
    if power > max_power:
        max_power = power
        max_x = x
        max_y = y
        max_size = size
    
(max_x, max_y, max_size)

(234, 108, 16)

# Day 12

## Part 1

In [192]:
with open('inputs/day12.txt') as f:
    s = f.read()[:-1].split('\n')
    
init = s[0].replace('initial state: ', '')
rules = {r.split(' => ')[0]: r.split(' => ')[1] for r in s[2:]}

def surroundings(state,i):
    l = len(state)
    if i == 0:
        return '..' + state[:3]
    if i == 1:
        return '.' + state[:4]
    if i == l-1:
        return state[l-3:] + '..'
    if i == l-2:
        return state[l-4:] + '.'
    return state[i-2:i+3]

def next_state(state, start):
    new_state = '..' + state + '..'
    start -= 2
    new_state = ''.join([rules[surroundings(new_state, i)] for i in range(len(new_state))])
    first_plant = new_state.index('#')
    start += first_plant
    return new_state[first_plant:new_state.rindex('#')+1], start

state = init
start = 0
generations = 20
for i in range(generations):
    state, start = next_state(state, start)
    
def plants(state, start):
    a = 0
    for i in range(len(state)):
        if state[i] == '#':
            a += start+i
    return a

plants(state, start)

3258

## Part 2

In [213]:
generations = 50000000000
stability_limit = 100
last_diff = 0
last_sum = 0
stability = 0

state = init
start = 0
for i in range(generations):
    state, start = next_state(state, start)
    a = plants(state, start)
    diff = a - last_sum
    if diff == last_diff:
        stability += 1
    last_diff = diff
    last_sum = a
    if stability >= stability_limit:
        break
    
last_sum + last_diff*(generations-i-1)

3600000002022

# Day 13

## Part 1

In [345]:
with open('inputs/day13.txt') as f:
    s = f.read()[:-1]
    
tracks = s.split('\n')
left = {(0,-1): (-1,0), (-1,0): (0,1), (0,1): (1,0), (1,0): (0,-1)}
right = {left[d]: d for d in left}
directions = {'<': (-1,0), '>': (1,0), '^': (0,-1), 'v': (0,1)}
turning_order = [lambda d: left[d], lambda d: d, lambda d: right[d]]
    
def cart_at(x,y,carts):
    return [c for c in carts if carts[c].x == x and carts[c].y == y]
    
class Cart:
    def __init__(self,cart_id,x,y,direction):
        self.x = x
        self.y = y
        self.direction = direction
        self.state = 0
        self.cart_id = cart_id
        
    def __repr__(self):
        return '({}, {})'.format(self.x, self.y)
        
    def crashed(self,carts):
        carts_here = cart_at(self.x, self.y,carts)
        return carts_here if len(carts_here) > 1 else []
        
    def move(self):
        self.x += self.direction[0]
        self.y += self.direction[1]
        if tracks[self.y][self.x] == '\\' and self.direction[0] == 0:
            self.direction = left[self.direction]
        elif tracks[self.y][self.x] == '\\' and self.direction[1] == 0:
            self.direction = right[self.direction]
        elif tracks[self.y][self.x] == '/' and self.direction[0] == 0:
            self.direction = right[self.direction]
        elif tracks[self.y][self.x] == '/' and self.direction[1] == 0:
            self.direction = left[self.direction]
        elif tracks[self.y][self.x] == '+':
            self.direction = turning_order[self.state](self.direction)
            self.state = (self.state + 1) % 3
        
        
carts = {}
for y in range(len(tracks)):
    for x in range(len(tracks[0])):
        if tracks[y][x] in '<>^v':
            carts[len(carts)] = Cart(len(carts),x,y,directions[tracks[y][x]])
    tracks[y] = tracks[y].replace('^', '|').replace('v', '|').replace('<','-').replace('>','-')
    
safe = True
def move_carts():
    global safe
    cart_order = [(cart, carts[cart].x, carts[cart].y) for cart in carts]
    cart_order.sort(key=lambda cart: cart[2])
    cart_order.sort(key=lambda cart: cart[1])
    
    crashed = []
    carts_new = {c:carts[c] for c in carts}
    for cart in [c[0] for c in cart_order]:
        if cart not in crashed:
            carts_new[cart].move()
            crash = carts_new[cart].crashed(carts_new)
            if len(crash) != 0 and safe:
                print(carts_new[cart].x, carts_new[cart].y)
                safe = False
            crashed += crash
            carts_new = {c: carts_new[c] for c in carts_new if c not in crashed}
    return carts_new

while safe:
    carts = move_carts()

123 18


## Part 2

In [346]:
tracks = s.split('\n')
carts = {}
for y in range(len(tracks)):
    for x in range(len(tracks[0])):
        if tracks[y][x] in '<>^v':
            carts[len(carts)] = Cart(len(carts),x,y,directions[tracks[y][x]])
    tracks[y] = tracks[y].replace('^', '|').replace('v', '|').replace('<','-').replace('>','-')

while len(carts) != 1:
    carts = move_carts()

carts

{2: (71, 123)}

# Day 14

## Part 1

In [360]:
recipe_count = 440231
recipes = ['3','7']
e0 = 0
e1 = 1

def create(recipes, e0, e1):
    new_recipe = int(recipes[e0])+int(recipes[e1])
    if new_recipe < 10:
        recipes.append(str(new_recipe))
    else:
        recipes.append(str(new_recipe//10))
        recipes.append(str(new_recipe%10))
    e0 = (e0 + int(recipes[e0]) + 1) % len(recipes)
    e1 = (e1 + int(recipes[e1]) + 1) % len(recipes)
    return recipes, e0, e1

while len(recipes) < recipe_count + 10:
    recipes, e0, e1 = create(recipes, e0, e1)
    
''.join(recipes[recipe_count:recipe_count+10])

'1052903161'

## Part 2

In [378]:
recipe_count_str = str(recipe_count)
recipes = ['3','7']
e0 = 0
e1 = 1

while recipe_count_str not in ''.join(recipes[-10:]):
    recipes, e0, e1 = create(recipes, e0, e1)
    
''.join(recipes).index(recipe_count_str)

20165504